In [1]:
import gspread
import os
import sys
import itertools

import numpy as np
import pandas as pd

In [2]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet_train = gc.open_by_key('1X-P5-djW3Q2hcm_G01zkouCzDseuKLwTHXNX0dpNWdI').sheet1
rows_train = worksheet_train.get_all_values()
df_train = pd.DataFrame(rows_train[1:], columns=rows_train[0])
# handle bool TRUE/FALSE to 1/0
df_train['is_recommended'] = df_train['is_recommended'].map({'TRUE': 1, 'FALSE': 0})
data_train = list(map(tuple, df_train.itertuples(index=False, name=None)))

print(f"Size of Data Train: {len(data_train)}")

print(f"Sample of Data Train: {data_train[:3]}")

Size of Data Train: 1007681
Sample of Data Train: [('136433', '6060', 1), ('2189692', '233980', 1), ('2387112', '204360', 1)]


In [3]:
!pip install --quiet cornac==2.3.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 35.7 MB/s eta 0:00:00


In [4]:
!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.1/cu121/repo.html

Looking in links: https://data.dgl.ai/wheels/torch-2.1/cu121/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.3/483.3 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 124.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176

In [5]:
import cornac
import dgl

print(f"System version: {sys.version}")
print(f"Cornac version: {cornac.__version__}")

SEED = 42
VERBOSE = True

test_size = 0.2

# for metrics, k = The number of items in the top@k list
k_top = 10

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
System version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
Cornac version: 2.3.2


In [6]:
for i in range(len(data_train)):
  data_train[i] = (int(data_train[i][0]), int(data_train[i][1]), int(data_train[i][2]))

In [7]:
worksheet_output = gc.open_by_key('1gqq8Xm3RGXgcuuUBKrW2_IgXLtKPpSeEsot1TuIb0U4').sheet1

headers = worksheet_output.row_values(1)

In [8]:
list_results_total = []

# Predictions will be evaluated in terms of ranking, NCRR@50, NDCG@50 and Recall@50.
# k = The number of items in the top@k list
eval_metrics = [
  cornac.metrics.NDCG(k=k_top),
  cornac.metrics.NCRR(k=k_top),
  cornac.metrics.Recall(k=k_top),
]

rs = cornac.eval_methods.RatioSplit(data_train, test_size=test_size, seed=SEED, verbose=VERBOSE)

rating_threshold = 1.0
exclude_unknowns = True


/usr/local/lib/python3.11/dist-packages/cornac/data/dataset.py:335: UserWarning: 2 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Training data:
Number of users = 389629
Number of items = 34638
Number of ratings = 806142
Max rating = 1.0
Min rating = 0.0
Global mean = 0.8
---
Test data:
Number of users = 389629
Number of items = 34638
Number of ratings = 130563
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 389629
Total items = 34638


In [11]:

# Define models
models = [
    # cornac.models.BaselineOnly(seed=SEED, name=f"Baseline Only"),
    # cornac.models.EASE(verbose=VERBOSE, seed=SEED, name=f"EASE"),
    # cornac.models.BPR(k=50, max_iter=100, learning_rate=0.01, lambda_reg=0.001, verbose=VERBOSE, seed=SEED, name=f"BPR(K=50)"),
    cornac.models.NeuMF(num_factors=50, layers=[64, 32, 16, 8], verbose=VERBOSE, seed=SEED, name=f"NeuMF", backend="pytorch"),
    # cornac.models.LightGCN(emb_size=64, num_layers=3, verbose=VERBOSE, seed=SEED, name=f"LightGCN")
]

In [12]:
for model in models:
  # Run experiment
  cornac.Experiment(
      eval_method=rs,
      models=[model],
      metrics=eval_metrics,
      user_based=True
  ).run()


[NeuMF] Training started!


  0%|          | 0/20 [00:00<?, ?it/s]


[NeuMF] Evaluation started!


Ranking:   0%|          | 0/66898 [00:00<?, ?it/s]


TEST:
...
      | NCRR@10 | NDCG@10 | Recall@10 | Train (s) | Test (s)
----- + ------- + ------- + --------- + --------- + --------
NeuMF |  0.0040 |  0.0060 |    0.0126 | 1156.5822 | 137.5957

